<a href="https://colab.research.google.com/github/gc2321/LangChain-Python/blob/main/4_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
!pip install --upgrade --quiet langchain-openai langchain-community
!pip install -quiet openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00

Usage:   
  pip3 install [op

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
open_ai_api ="/content/gdrive/MyDrive/llm/api_key.txt"
f = open(open_ai_api)
api_key = f.read()

In [5]:
os.environ['OPENAI_API_KEY'] = api_key

### Read Email

In [6]:
path = "/content/gdrive/MyDrive/llm/langchain/some_data/spanish_customer_email.txt"

In [10]:
# Open the file with the correct encoding
with open(path, 'r', encoding='latin-1') as f:
    spanish_email = f.read()

In [11]:
print(spanish_email)

Asunto: Reporte de Problemas Técnicos - Funcionalidad del Panel SAAS

Estimado Equipo de Soporte al Cliente,

Espero que este mensaje les encuentre bien. Les escribo para informarles sobre un problema técnico que he encontrado mientras utilizo su producto de panel SAAS. Como cliente leal, aprecio el valor que su producto aporta a mi negocio, pero actualmente me enfrento a un desafío que requiere su experiencia.

Me gustaría describir detalladamente el problema que estoy experimentando:

1. Problema Gráfico: Al iniciar sesión en el panel SAAS, he notado que los gráficos y las tablas en la página principal del panel no se renderizan correctamente. Los puntos de datos aparecen distorsionados y algunos elementos se superponen, lo que dificulta la interpretación precisa de la información.

2. Fallo en la Función de Exportación: Además, no he podido exportar informes y datos desde el panel. Cada vez que intento exportar un informe en formato CSV o PDF, recibo un mensaje de error que indica q

In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [13]:
def translate_and_summarize(email):
    """
    Translates an email written in a detected language to English and generates a summary.

    Args:
        email (str): The email to be processed and translated.

    Returns:
        dict: A dictionary containing the following keys:
            - 'language': The language the email was written in.
            - 'translated_email': The translated version of the email in English.
            - 'summary': A short summary of the translated email.

    Raises:
        Exception: If any error occurs during the LLM chain execution.

    Example:
        email = "Hola, ¿cómo estás? Espero que todo vaya bien."
        result = translate_and_summarize(email)
        print(result)
        # Output:
        # {
        #     'language': 'Spanish',
        #     'translated_email': 'Hello, how are you? I hope everything is going well.',
        #     'summary': 'A friendly greeting and a wish for well-being.'
        # }
    """
    # Create Model
    llm = ChatOpenAI()

    # CREATE A CHAIN THAT DOES THE FOLLOWING:

    # Detect Language
    template1 = "Return the language this email is written in:\n{email}.\nONLY return the language it was written in."
    prompt1 = ChatPromptTemplate.from_template(template1)
    chain_1 = prompt1|llm

    # Translate from detected language to English
    template2 = "Translate this email from {language} to English. Here is the email:\n"+email
    prompt2 = ChatPromptTemplate.from_template(template2)
    chain_2 = prompt2|llm

    # Return English Summary AND the Translated Email
    template3 = "Create a short summary of this email:\n{translated_email}"
    prompt3 = ChatPromptTemplate.from_template(template3)
    chain_3 = prompt3|llm

    language_chain = chain_1
    translation_chain = language_chain|chain_2
    seq_chain = translation_chain|chain_3

    return language_chain.invoke(email), translation_chain.invoke(email), seq_chain.invoke(email)

In [14]:
result = translate_and_summarize(spanish_email)

In [15]:
result

(AIMessage(content='Spanish', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 826, 'total_tokens': 827}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1297bbc8-ae3e-474a-8226-f0f59e25f445-0', usage_metadata={'input_tokens': 826, 'output_tokens': 1, 'total_tokens': 827}),
 AIMessage(content="Subject: Technical Issues Report - SAAS Panel Functionality\n\nDear Customer Support Team,\n\nI hope this message finds you well. I am writing to inform you about a technical issue I have encountered while using your SAAS panel product. As a loyal customer, I appreciate the value your product brings to my business, but I am currently facing a challenge that requires your expertise.\n\nI would like to describe in detail the problem I am experiencing:\n\n1. Graphic Issue: When logging into the SAAS panel, I have noticed that the graphs and tables on the main panel page are not rendering correctly. Data points 

In [16]:
result[0]

AIMessage(content='Spanish', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 826, 'total_tokens': 827}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1297bbc8-ae3e-474a-8226-f0f59e25f445-0', usage_metadata={'input_tokens': 826, 'output_tokens': 1, 'total_tokens': 827})

In [17]:
result[1]

AIMessage(content="Subject: Technical Issues Report - SAAS Panel Functionality\n\nDear Customer Support Team,\n\nI hope this message finds you well. I am writing to inform you about a technical issue I have encountered while using your SAAS panel product. As a loyal customer, I appreciate the value your product brings to my business, but I am currently facing a challenge that requires your expertise.\n\nI would like to describe in detail the problem I am experiencing:\n\n1. Graphic Issue: When logging into the SAAS panel, I have noticed that the graphs and tables on the main panel page are not rendering correctly. Data points appear distorted and some elements overlap, making it difficult to accurately interpret the information.\n\n2. Export Function Failure: Additionally, I have been unable to export reports and data from the panel. Every time I try to export a report in CSV or PDF format, I receive an error message indicating that the export has failed. This functionality is crucial 

In [18]:
result[2]

AIMessage(content='This email outlines technical issues with a SAAS panel product, including graphical rendering problems, export function failures, and slow loading speeds. The customer has already attempted troubleshooting steps and requests assistance from the customer support team to resolve these issues promptly. They provide specific actions they would like to see taken to improve the functionality of the SAAS panel.', response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 739, 'total_tokens': 806}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d423672b-caea-487f-91e0-6a41f09932a1-0', usage_metadata={'input_tokens': 739, 'output_tokens': 67, 'total_tokens': 806})